<a href="https://colab.research.google.com/github/raki-rankawat/stm32/blob/main/VWW_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time
import tarfile
import random
import shutil
from pathlib import Path
from urllib.request import urlretrieve

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ----------------------------
# Auto Download + Prepare VWW (10k subset)
# ----------------------------
vww_url = "https://www.silabs.com/public/files/github/machine_learning/benchmarks/datasets/vw_coco2014_96.tar.gz"

base_dir = Path("/content/vww_work")
archive_path = base_dir / "vw_coco2014_96.tar.gz"
extract_dir = base_dir / "extracted"
subset_dir = base_dir / "vww_10k"

# Subset config: 5k person + 5k non_person
n_per_class = 5000
val_ratio = 0.20

random.seed(41)
torch.manual_seed(41)

def download_vww():
    base_dir.mkdir(parents=True, exist_ok=True)

    if archive_path.exists() and archive_path.stat().st_size > 0:
        print("✅ VWW archive already downloaded")
        return

    print("⬇️ Downloading VWW archive...")
    urlretrieve(vww_url, archive_path)
    print("✅ Download complete:", archive_path)

def extract_vww():
    extract_dir.mkdir(parents=True, exist_ok=True)

    if any(extract_dir.iterdir()):
        print("✅ VWW already extracted")
        return

    print("📦 Extracting VWW archive...")
    with tarfile.open(archive_path, "r:gz") as tar:
        tar.extractall(extract_dir)
    print("✅ Extraction complete:", extract_dir)

def find_vww_root():
    # Find folder that contains BOTH person/ and non_person/
    for p in extract_dir.rglob("person"):
        if p.is_dir() and (p.parent / "non_person").is_dir():
            return p.parent
    raise RuntimeError("❌ Could not find 'person' and 'non_person' directories under extracted dataset")

def list_images(folder):
    exts = {".jpg", ".jpeg", ".png"}
    return [p for p in folder.rglob("*") if p.is_file() and p.suffix.lower() in exts]

def make_vww_subset(src_root):
    # Skip if subset already exists
    if (subset_dir / "train" / "person").is_dir() and (subset_dir / "val" / "non_person").is_dir():
        print("✅ VWW 10k subset already exists:", subset_dir)
        return

    for split in ["train", "val"]:
        for c in ["person", "non_person"]:
            (subset_dir / split / c).mkdir(parents=True, exist_ok=True)

    person_imgs = list_images(src_root / "person")
    nonperson_imgs = list_images(src_root / "non_person")

    if len(person_imgs) < n_per_class or len(nonperson_imgs) < n_per_class:
        raise ValueError(
            f"❌ Not enough images:\n"
            f"person: {len(person_imgs)} (need {n_per_class})\n"
            f"non_person: {len(nonperson_imgs)} (need {n_per_class})"
        )

    random.shuffle(person_imgs)
    random.shuffle(nonperson_imgs)

    person_sel = person_imgs[:n_per_class]
    nonperson_sel = nonperson_imgs[:n_per_class]

    def split_list(lst, val_ratio):
        n_val = int(len(lst) * val_ratio)
        return lst[n_val:], lst[:n_val]  # train, val

    p_train, p_val = split_list(person_sel, val_ratio)
    n_train, n_val = split_list(nonperson_sel, val_ratio)

    def copy_files(files, dst_dir):
        for f in files:
            dst = dst_dir / f.name
            # avoid rare collisions
            if dst.exists():
                dst = dst_dir / (f"{f.parent.name}_{f.name}")
            shutil.copy2(f, dst)

    print("🧩 Creating VWW 10k subset...")
    copy_files(p_train, subset_dir / "train" / "person")
    copy_files(p_val,   subset_dir / "val"   / "person")
    copy_files(n_train, subset_dir / "train" / "non_person")
    copy_files(n_val,   subset_dir / "val"   / "non_person")
    print("✅ VWW subset created at:", subset_dir)

download_vww()
extract_vww()
vww_root = find_vww_root()
print("✅ Found VWW root:", vww_root)
make_vww_subset(vww_root)

⬇️ Downloading VWW archive...
✅ Download complete: /content/vww_work/vw_coco2014_96.tar.gz
📦 Extracting VWW archive...


/tmp/ipython-input-4249398104.py:38: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(extract_dir)


✅ Extraction complete: /content/vww_work/extracted
✅ Found VWW root: /content/vww_work/extracted/vw_coco2014_96
🧩 Creating VWW 10k subset...
✅ VWW subset created at: /content/vww_work/vww_10k


In [ ]:
# ----------------------------
# Data Loaders (same style)
# ----------------------------
batch_size = 64
img_size = 96

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(img_size, scale=(0.6, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

test_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

train_data = datasets.ImageFolder(root=str(subset_dir / "train"), transform=train_transform)
test_data  = datasets.ImageFolder(root=str(subset_dir / "val"),   transform=test_transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_data,  batch_size=batch_size, shuffle=False)

print("Class mapping:", train_data.class_to_idx)

Class mapping: {'non_person': 0, 'person': 1}


In [ ]:
# ----------------------------
# CNN Model (same style) - FINAL
# ----------------------------
class VWWConvNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)

        # For 96x96 with 4 pools: 96->48->24->12->6
        self.fc1 = nn.Linear(128 * 6 * 6, 256)
        self.fc2 = nn.Linear(256, 2)

        self.dropout = nn.Dropout(0.30)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, 2)

        x = x.view(x.size(0), -1)

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

In [ ]:
# ----------------------------
# Random Seeds and Model Instance (same style)
# ----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(41)
model = VWWConvNet().to(device)

In [ ]:
# ----------------------------
# Loss & Optimizer (same style) - FINAL
# ----------------------------
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)

In [ ]:

# ----------------------------
# Training (same style)
# ----------------------------
def train(model, loader, criterion, optimizer, device):
    model.train() # training mode

    correct = 0
    total = 0
    running_loss = 0

    for X, y in loader:
        X = X.to(device)
        y = y.to(device)

        # Forward
        outputs = model(X)

        # Loss
        loss = criterion(outputs, y)

        # Backward
        optimizer.zero_grad(set_to_none = True)
        loss.backward()
        optimizer.step()

        # Statistics
        batch_size = y.size(0)
        running_loss += loss.item() * batch_size
        preds = outputs.argmax(dim = 1)
        correct += (preds == y).sum().item()
        total += batch_size

    avg_loss = running_loss / total
    accuracy = correct / total

    return avg_loss, accuracy

In [ ]:
# ----------------------------
# Testing (same style)
# ----------------------------
def test(model, loader, criterion, device):
    model.eval() # testing mode

    correct = 0
    total = 0
    running_loss = 0

    with torch.no_grad():
        for X, y in loader:
            X = X.to(device)
            y = y.to(device)

            # Forward
            outputs = model(X)

            # Loss
            loss = criterion(outputs, y)

            # Statistics
            batch_size = y.size(0)
            running_loss += loss.item() * batch_size
            preds = outputs.argmax(dim = 1)
            correct += (preds == y).sum().item()
            total += batch_size

    avg_loss = running_loss / total
    accuracy = correct / total

    return avg_loss, accuracy

In [ ]:
# ----------------------------
# Run (same style print) - FINAL (best checkpoint + early stopping)
# ----------------------------
epochs = 50
start_time = time.time()

best_acc = 0.0
best_epoch = 1
patience = 8
patience_counter = 0

best_path = "/content/drive/My Drive/Colab Notebooks/stm_vww_best.pth"

for epoch in range(1, epochs + 1):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = test(model, test_loader, criterion, device)

    print(
        f"Epoch: {epoch}/{epochs} | "
        f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc * 100:.2f}% | "
        f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc * 100:.2f}%"
    )

    if test_acc > best_acc:
        best_acc = test_acc
        best_epoch = epoch
        patience_counter = 0
        torch.save(model.state_dict(), best_path)
        print("✅ Model saved as stm_vww_best.pth for epoch", best_epoch)
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("🛑 Early stopping at epoch", epoch, "| Best epoch:", best_epoch, "| Best Test Acc:", f"{best_acc*100:.2f}%")
        break

print(f"Training Time: {(time.time() - start_time) / 60} minutes!")
print("Best epoch:", best_epoch, "| Best Test Acc:", f"{best_acc*100:.2f}%")

Epoch: 1/50 | Train Loss: 0.6790 | Train Acc: 60.89% | Test Loss: 0.6175 | Test Acc: 66.10%
✅ Model saved as stm_vww_best.pth for epoch 1
Epoch: 2/50 | Train Loss: 0.6309 | Train Acc: 65.03% | Test Loss: 0.6353 | Test Acc: 62.60%
Epoch: 3/50 | Train Loss: 0.6075 | Train Acc: 67.74% | Test Loss: 0.5751 | Test Acc: 69.50%
✅ Model saved as stm_vww_best.pth for epoch 3
Epoch: 4/50 | Train Loss: 0.5854 | Train Acc: 68.75% | Test Loss: 0.6575 | Test Acc: 60.25%
Epoch: 5/50 | Train Loss: 0.5795 | Train Acc: 69.91% | Test Loss: 0.5508 | Test Acc: 73.15%
✅ Model saved as stm_vww_best.pth for epoch 5
Epoch: 6/50 | Train Loss: 0.5667 | Train Acc: 70.99% | Test Loss: 0.5605 | Test Acc: 71.80%
Epoch: 7/50 | Train Loss: 0.5560 | Train Acc: 72.32% | Test Loss: 0.5762 | Test Acc: 71.90%
Epoch: 8/50 | Train Loss: 0.5471 | Train Acc: 73.08% | Test Loss: 0.5462 | Test Acc: 73.85%
✅ Model saved as stm_vww_best.pth for epoch 8
Epoch: 9/50 | Train Loss: 0.5418 | Train Acc: 72.81% | Test Loss: 0.6337 | Test 